In [1]:
from pyspark.sql import SparkSession
import time 


# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.197:7077") \
        .appName("Test_wordcount")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/15 07:25:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/15 07:25:39 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [2]:
startTimeQuery = time.time()
path = "hdfs://130.238.28.151:9000/user/ubuntu/input/20417.txt.utf-8"
data = spark_context.textFile(path)
count = data.count()
print(count)

#Pre-process the text 
    #Lowercase the text
    #Tokenize the text (split on space)
    
lowercased = data.map(lambda line: line.lower())
tokenized = lowercased.flatMap(lambda word: word.split(' '))

mapped = tokenized.map(lambda w: w.strip())\
    .map(lambda w: (w,1))

merged = mapped.reduceByKey(lambda x,y: x+y)

print(merged.takeOrdered(10, key=lambda x: -x[1]))


endTimeQuery = time.time()
runTimeQuery = round((endTimeQuery - startTimeQuery),3)
print(f"{runTimeQuery} seconds")

12760


[('', 14639), ('the', 9154), ('of', 5754), ('and', 2909), ('a', 2728), ('in', 2347), ('to', 2233), ('is', 2098), ('it', 1322), ('that', 1312)]
11.097 seconds


In [3]:
spark_context.stop()